# Supplementary Results 8
## Sample-by-Sample Detection Evaluation 

In [1]:
import copy

import pandas as pd
from plotly.subplots import make_subplots
import plotly.io as pio

from analysis._article_results.lund2013._helpers import *
import analysis.statistics.sample_metrics as sm

pio.renderers.default = "browser"

C:\Users\jonathanni\PycharmProjects\pEYES\venv\Lib\site-packages\statsmodels\tools\tools.py:6: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 2.3.0)
  import scipy.linalg


### Load Data

In [2]:
global_measures = sm.load_global_metrics(DATASET_NAME, PROCESSED_DATA_DIR, stimulus_type=STIMULUS_TYPE, metric=None, iteration=1)
global_measures.drop(index=[peyes.constants.ACCURACY_STR, peyes.constants.BALANCED_ACCURACY_STR], inplace=True)    # Drop Acc+Balanced-Acc metrics

fixation_sdt = sm.load_sdt(DATASET_NAME, PROCESSED_DATA_DIR, label=1, stimulus_type=STIMULUS_TYPE, metric=None, iteration=1)
fixation_sdt = fixation_sdt.loc[[peyes.constants.D_PRIME_STR, peyes.constants.F1_STR]]      # Keep only d' and f1 metrics
fixation_sdt = fixation_sdt.rename(index=lambda idx: f"fixation_{idx}")     # Rename index

saccade_sdt = sm.load_sdt(DATASET_NAME, PROCESSED_DATA_DIR, label=2, stimulus_type=STIMULUS_TYPE, metric=None)
saccade_sdt = saccade_sdt.loc[[peyes.constants.D_PRIME_STR, peyes.constants.F1_STR]]        # Keep only d' and f1 metrics
saccade_sdt = saccade_sdt.rename(index=lambda idx: f"saccade_{idx}")        # Rename index

fixation_saccade_sdt = pd.concat([fixation_sdt, saccade_sdt], axis=0)

### Fixation & Saccade Detection
Evaluate performance by measuring how well the detector detects _fixation-samples_ or _saccade-samples_ out of all samples.  
Evaluation is based on _Discriminability Index_ ($d'$) and _f1-score_

In [3]:
sdt_statistics, sdt_pvalues, sdt_nemenyi, sdt_Ns = sm.friedman_nemenyi(fixation_saccade_sdt, [GT1, GT2])

sdt_pvalues <= ALPHA

gt,MN,RA
metric,,
fixation_d_prime,True,True
fixation_f1,True,True
saccade_d_prime,True,True
saccade_f1,True,True


In [4]:
pd.concat([sdt_statistics, sdt_pvalues], axis=1, keys=['Q', 'p']).stack(1, future_stack=True)

Q             p
metric           gt                         
fixation_d_prime MN  53.953548  7.537991e-10
                 RA  70.502712  3.224626e-13
fixation_f1      MN  66.325161  2.312691e-12
                 RA  91.095841  1.793744e-17
saccade_d_prime  MN  56.205656  2.645147e-10
                 RA  78.646900  6.798216e-15
saccade_f1       MN  67.156812  1.563243e-12
                 RA  97.428571  8.625070e-19

#### Post Hoc Analysis

In [5]:
post_hoc_fix_dprime = sm.post_hoc_table(sdt_nemenyi, f"fixation_{peyes.constants.D_PRIME_STR}", [GT1, GT2], alpha=ALPHA, marginal_alpha=MARGINAL_ALPHA)
post_hoc_fix_dprime

pred            ivt    ivvt     idt    idvt engbert      nh remodnav
pred     gt                                                         
ivt      MN      --    n.s.       *       †    n.s.    n.s.        *
         RA      --    n.s.       *    n.s.    n.s.       *        *
ivvt     MN  0.9982      --    n.s.    n.s.    n.s.    n.s.     n.s.
         RA  0.9751      --    n.s.    n.s.    n.s.    n.s.     n.s.
idt      MN  0.0144  0.0918      --    n.s.       †    n.s.     n.s.
         RA  0.0383  0.3756      --    n.s.    n.s.    n.s.     n.s.
idvt     MN  0.0669  0.2727  0.9995      --    n.s.    n.s.     n.s.
         RA  0.1333  0.6661  0.9996      --    n.s.    n.s.     n.s.
engbert  MN  0.9997  1.0000  0.0606  0.2016      --    n.s.        †
         RA  0.9950  1.0000  0.2389  0.5027      --    n.s.     n.s.
nh       MN  0.1390  0.4362  0.9930  1.0000  0.3442      --     n.s.
         RA  0.0286  0.3221  1.0000  0.9989  0.1977      --     n.s.
remodnav MN  0.0137  0.0884  1.0000  0.9994  0.0582  0.9923       --
         RA  0.0120  0.1977  0.9999  0.9912  0.1099  1.0000       --

In [6]:
post_hoc_fix_f1 = sm.post_hoc_table(sdt_nemenyi, f"fixation_{peyes.constants.F1_STR}", [GT1, GT2], alpha=ALPHA, marginal_alpha=MARGINAL_ALPHA)
post_hoc_fix_f1

pred            ivt    ivvt     idt    idvt engbert      nh remodnav
pred     gt                                                         
ivt      MN      --    n.s.    n.s.    n.s.    n.s.    n.s.       **
         RA      --    n.s.    n.s.    n.s.    n.s.    n.s.      ***
ivvt     MN  1.0000      --    n.s.    n.s.    n.s.    n.s.      ***
         RA  0.9999      --    n.s.    n.s.    n.s.    n.s.      ***
idt      MN  0.3733  0.2940      --    n.s.       †    n.s.     n.s.
         RA  0.2850  0.1411      --    n.s.       †    n.s.     n.s.
idvt     MN  0.5017  0.4124  1.0000      --    n.s.    n.s.     n.s.
         RA  0.3756  0.2020  1.0000      --    n.s.    n.s.     n.s.
engbert  MN  0.9873  0.9951  0.0539  0.0947      --    n.s.      ***
         RA  0.9948  0.9999  0.0501  0.0790      --    n.s.      ***
nh       MN  0.9293  0.8841  0.9682  0.9900  0.4771      --     n.s.
         RA  0.9463  0.8307  0.9174  0.9570  0.6074      --        †
remodnav MN  0.0015  0.0008  0.6268  0.4941  0.0000  0.1013       --
         RA  0.0010  0.0002  0.6619  0.5585  0.0000  0.0642       --

In [7]:
post_hoc_sac_dprime = sm.post_hoc_table(sdt_nemenyi, f"saccade_{peyes.constants.D_PRIME_STR}", [GT1, GT2], alpha=ALPHA, marginal_alpha=MARGINAL_ALPHA)
post_hoc_sac_dprime

pred            ivt    ivvt     idt    idvt engbert      nh remodnav
pred     gt                                                         
ivt      MN      --    n.s.    n.s.    n.s.    n.s.    n.s.     n.s.
         RA      --    n.s.    n.s.    n.s.    n.s.    n.s.        †
ivvt     MN  1.0000      --    n.s.    n.s.    n.s.    n.s.     n.s.
         RA  1.0000      --    n.s.    n.s.    n.s.    n.s.        †
idt      MN  0.7356  0.7422      --    n.s.      **    n.s.      ***
         RA  0.4640  0.4728      --    n.s.     ***    n.s.      ***
idvt     MN  0.8729  0.8774  1.0000      --       *    n.s.      ***
         RA  0.7657  0.7731  0.9995      --     ***    n.s.      ***
engbert  MN  0.5036  0.4960  0.0073  0.0201      --    n.s.     n.s.
         RA  0.1717  0.1665  0.0001  0.0007      --    n.s.     n.s.
nh       MN  1.0000  1.0000  0.7322  0.8706  0.5074      --     n.s.
         RA  0.9999  0.9999  0.2513  0.5373  0.3514      --     n.s.
remodnav MN  0.0925  0.0898  0.0002  0.0006  0.9863  0.0940       --
         RA  0.0617  0.0594  0.0000  0.0001  0.9998  0.1589       --

In [8]:
post_hoc_sac_f1 = sm.post_hoc_table(sdt_nemenyi, f"saccade_{peyes.constants.F1_STR}", [GT1, GT2], alpha=ALPHA, marginal_alpha=MARGINAL_ALPHA)
post_hoc_sac_f1

pred            ivt    ivvt     idt    idvt engbert      nh remodnav
pred     gt                                                         
ivt      MN      --    n.s.       *       †    n.s.    n.s.     n.s.
         RA      --    n.s.      **      **    n.s.    n.s.     n.s.
ivvt     MN  1.0000      --       †    n.s.    n.s.    n.s.     n.s.
         RA  1.0000      --      **      **    n.s.    n.s.     n.s.
idt      MN  0.0419  0.0508      --    n.s.       †    n.s.      ***
         RA  0.0023  0.0025      --    n.s.      **    n.s.      ***
idvt     MN  0.0632  0.0756  1.0000      --    n.s.    n.s.      ***
         RA  0.0039  0.0041  1.0000      --      **    n.s.      ***
engbert  MN  1.0000  1.0000  0.0517  0.0768      --    n.s.     n.s.
         RA  1.0000  1.0000  0.0015  0.0026      --    n.s.     n.s.
nh       MN  0.6619  0.7012  0.8658  0.9158  0.7047      --       **
         RA  0.4455  0.4542  0.6152  0.6945  0.3860      --      ***
remodnav MN  0.5226  0.4809  0.0000  0.0000  0.4771  0.0051       --
         RA  0.2709  0.2640  0.0000  0.0000  0.3221  0.0002       --

## Original Figure

In [9]:
sdt_metrics_fig = sm.sdt_distributions_figure(
    fixation_saccade_sdt,
    GT1, GT2,
    colors={k: v[1] for k, v in LABELER_PLOTTING_CONFIG.items()},
    only_box=False,
    show_other_gt=True,
    share_x=True,
)

sdt_metrics_fig.show()

## Final Fig:
#### (1) Only Show d'
#### (2) Show RA and MN Separately

In [10]:
NAME = "supp_fig_E1"
WIDTH, HEIGHT = 1200, 800

ROW_TITLES, COL_TITLES = ["GT: <i>RA</i>", "GT: <i>MN</i>"], ["Fixations", "Saccades"]

FONT_FAMILY, FONT_COLOR = "Calibri", "black"
TITLE_FONT = dict(family=FONT_FAMILY, size=25, color=FONT_COLOR)
AXIS_LABEL_FONT = dict(family=FONT_FAMILY, size=22, color=FONT_COLOR)
AXIS_TICK_FONT = dict(family=FONT_FAMILY, size=18, color=FONT_COLOR)

In [11]:
fig = make_subplots(
    rows=len(ROW_TITLES), cols=len(COL_TITLES),
    row_titles=ROW_TITLES, column_titles=COL_TITLES,
    shared_xaxes=True, shared_yaxes=True,
    vertical_spacing=0.05, horizontal_spacing=0.05,
)

for trace in sdt_metrics_fig.data:
    if trace["scalegroup"].endswith('f1'):
        continue
    new_tr = copy.deepcopy(trace)
    gt1 = new_tr["name"].split(",")[0]
    gt2 = "MN" if gt1=="RA" else "RA"

    # convert from violin plot to ridge plot by assigning the `y0` prop and removing `x0` prop
    if new_tr["x0"] == "Other GT":
        new_tr["y0"] = new_tr["name"] = new_tr["legendgroup"] = f"Ann. {gt2}"
    elif new_tr["x0"].startswith("i"):
        new_tr["y0"] = new_tr["name"] = new_tr["legendgroup"] = new_tr["x0"].replace("i", "I-").upper()
    elif new_tr["x0"] == "remodnav":
        new_tr["y0"] = new_tr["name"] = new_tr["legendgroup"] = "REMoDNaV"
    else:
        new_tr["y0"] = new_tr["name"] = new_tr["legendgroup"] = new_tr["x0"].upper()
    new_tr['x'] = new_tr['y']

    # reset irrelevant props
    new_tr['y'] = new_tr['x0'] = None
    new_tr["showlegend"] = new_tr['points'] = False
    new_tr["side"] = "positive"
    new_tr["box"] = None
    new_tr["meanline"] = dict(visible=True, width=3, color='lightgray')
    new_tr["width"] = 1.75
    new_tr["opacity"] = 1
    new_tr["visible"] = True
    new_tr["scalegroup"] = new_tr["scalegroup"].split("_")[0]

    # add the trace to the correct subplot
    r = 1 if gt1=="RA" else 2
    c = 1 if new_tr["scalegroup"].startswith("fixation") else 2
    fig.add_trace(new_tr, row=r, col=c)

# update x axes
fig.for_each_xaxis(lambda xax: xax.update(
    showline=False, zeroline=False, showgrid=False, gridcolor='lightgray', gridwidth=1, tickfont=AXIS_TICK_FONT,
))
for c in range(len(COL_TITLES)):
    fig.update_xaxes(row=2, col=c+1, title=dict(text=r"$d'$", font=AXIS_LABEL_FONT, standoff=2),)

# update y axes
fig.for_each_yaxis(lambda yax: yax.update(
    showline=False, zeroline=False, showgrid=True, gridcolor='lightgray', gridwidth=1, tickfont=AXIS_TICK_FONT,
))
for r in range(len(ROW_TITLES)):
    fig.update_yaxes(row=r+1, col=1, title=dict(text="Detector", font=AXIS_LABEL_FONT, standoff=4),)

# update subtitles
fig.for_each_annotation(lambda ann: ann.update(
    font=TITLE_FONT, textangle=0,
    xref='paper', xanchor='center', x=0.49 if ann.text in ROW_TITLES else ann.x,
    yref='paper', yanchor='top', y=1.0 if ann.text==ROW_TITLES[0] else 0.475 if ann.text==ROW_TITLES[1] else 1.05,
))

fig.update_layout(
    font_family=FONT_FAMILY,
    width=WIDTH, height=HEIGHT,
    paper_bgcolor='rgba(0, 0, 0, 0)', plot_bgcolor='rgba(0, 0, 0, 0)',
    margin=dict(l=0, r=0, t=27.5, b=45, pad=0),
    showlegend=False,
)

fig.write_image(os.path.join(FIGURES_DIR, f"{NAME}.png"), scale=3)
# fig.write_json(os.path.join(FIGURES_DIR, f"{NAME}.json"))
fig.show()

## Repeat Analysis
### Repeating the analysis with subset of image-stimulus trials that were recorded @ 500Hz

In [12]:
subset = fixation_saccade_sdt.drop(columns=NON_500HZ_TRIALS, level=peyes.constants.TRIAL_ID_STR)

subset_statistics, subset_pvalues, subset_nemenyi, subset_Ns = sm.friedman_nemenyi(fixation_saccade_sdt, [GT1, GT2])
pd.concat([sdt_statistics, sdt_pvalues, subset_pvalues <= ALPHA], axis=1, keys=['Q', 'p', 'is_sig']).stack(1, future_stack=True)

Q             p  is_sig
metric           gt                                 
fixation_d_prime MN  53.953548  7.537991e-10    True
                 RA  70.502712  3.224626e-13    True
fixation_f1      MN  66.325161  2.312691e-12    True
                 RA  91.095841  1.793744e-17    True
saccade_d_prime  MN  56.205656  2.645147e-10    True
                 RA  78.646900  6.798216e-15    True
saccade_f1       MN  67.156812  1.563243e-12    True
                 RA  97.428571  8.625070e-19    True

In [13]:
subset_post_hoc_fix_dprime = sm.post_hoc_table(subset_nemenyi, f"fixation_{peyes.constants.D_PRIME_STR}", [GT1, GT2], alpha=ALPHA, marginal_alpha=MARGINAL_ALPHA)
subset_post_hoc_fix_dprime

pred            ivt    ivvt     idt    idvt engbert      nh remodnav
pred     gt                                                         
ivt      MN      --    n.s.       *       †    n.s.    n.s.        *
         RA      --    n.s.       *    n.s.    n.s.       *        *
ivvt     MN  0.9982      --    n.s.    n.s.    n.s.    n.s.     n.s.
         RA  0.9751      --    n.s.    n.s.    n.s.    n.s.     n.s.
idt      MN  0.0144  0.0918      --    n.s.       †    n.s.     n.s.
         RA  0.0383  0.3756      --    n.s.    n.s.    n.s.     n.s.
idvt     MN  0.0669  0.2727  0.9995      --    n.s.    n.s.     n.s.
         RA  0.1333  0.6661  0.9996      --    n.s.    n.s.     n.s.
engbert  MN  0.9997  1.0000  0.0606  0.2016      --    n.s.        †
         RA  0.9950  1.0000  0.2389  0.5027      --    n.s.     n.s.
nh       MN  0.1390  0.4362  0.9930  1.0000  0.3442      --     n.s.
         RA  0.0286  0.3221  1.0000  0.9989  0.1977      --     n.s.
remodnav MN  0.0137  0.0884  1.0000  0.9994  0.0582  0.9923       --
         RA  0.0120  0.1977  0.9999  0.9912  0.1099  1.0000       --

In [14]:
subset_post_hoc_sac_dprime = sm.post_hoc_table(subset_nemenyi, f"saccade_{peyes.constants.D_PRIME_STR}", [GT1, GT2], alpha=ALPHA, marginal_alpha=MARGINAL_ALPHA)
subset_post_hoc_sac_dprime

pred            ivt    ivvt     idt    idvt engbert      nh remodnav
pred     gt                                                         
ivt      MN      --    n.s.    n.s.    n.s.    n.s.    n.s.     n.s.
         RA      --    n.s.    n.s.    n.s.    n.s.    n.s.        †
ivvt     MN  1.0000      --    n.s.    n.s.    n.s.    n.s.     n.s.
         RA  1.0000      --    n.s.    n.s.    n.s.    n.s.        †
idt      MN  0.7356  0.7422      --    n.s.      **    n.s.      ***
         RA  0.4640  0.4728      --    n.s.     ***    n.s.      ***
idvt     MN  0.8729  0.8774  1.0000      --       *    n.s.      ***
         RA  0.7657  0.7731  0.9995      --     ***    n.s.      ***
engbert  MN  0.5036  0.4960  0.0073  0.0201      --    n.s.     n.s.
         RA  0.1717  0.1665  0.0001  0.0007      --    n.s.     n.s.
nh       MN  1.0000  1.0000  0.7322  0.8706  0.5074      --     n.s.
         RA  0.9999  0.9999  0.2513  0.5373  0.3514      --     n.s.
remodnav MN  0.0925  0.0898  0.0002  0.0006  0.9863  0.0940       --
         RA  0.0617  0.0594  0.0000  0.0001  0.9998  0.1589       --